In [1]:
def ReduceCluster(S, eps=1*10**-6, c=1):
    n = S[0].parent().codomain().dimension_relative() + 1
    cluster0 = [vector(list(pnt)) for pnt in S]
    m = len(cluster0)
    C = ComplexField(prec = 2000)
    R = RealField(prec = 2000)
    for v in cluster0:
        v.change_ring(C)
    cluster = [v*1/R(v.norm()) for v in cluster0]
    Tr = matrix.identity(n)
    count = 0
    total_count = 0
    while total_count < S[1][1].prec()*2:
        total_count += 1
        cols = matrix(cluster).transpose()
        der1 = [R(col.hermitian_inner_product(col)) for col in cols]
        der2 = []
        for i in range(n):
            temp_list = []
            for j in range(i):
                temp_list.append(2*R(cols[i].hermitian_inner_product(cols[j])))
            der2.append(temp_list)
        average_der1 = sum(der1)/n
        der1 = [r - average_der1 for r in der1]
        absd = 0
        der2_prime = [] # unpack the list der2
        for lst in der2:
            der2_prime += lst
        for d in der1 + der2_prime:
            absd += d**2
        print('|der| =', absd)
        if absd < eps**2:
            break
        for i in range(len(der2)):
            der2[i].append(der1[i])
            
        der3 = []
        for lst in der2:
            new_lst = lst + [C(0)]*(n - len(lst))
            der3.append(new_lst)
            
        # make symmetric matrix
        B1 = matrix(der3)
        for i in range(n-1):
            for j in range(1 + i, n):
                B1[i, j] = B1[j, i]
        B = MatExp(-c*B1, eps=eps*1**-10).change_ring(R)
        B *= B.determinant()**(-1/n)
        Tr *= B
        cluster1 = [B*pnt for pnt in cluster]
        val = sum(pnt.hermitian_inner_product(pnt).log() for pnt in cluster1)
        print('gain =', val)
        if val > 0:
            c /= 2
            print('new c =', c)
        else:
            count += 1
            if count == 10:
                count = 0
                Tr *= R(Tr.det())**(-1/n)
                cluster1 = [pt*Tr for pt in cluster0]
            cluster = [v*1/R(v.norm()) for v in cluster1]
    Tr = Tr*Tr.transpose()
    Tr = Tr.change_ring(RealField(prec=100))
    Tr = (Tr + Tr.transpose())/2
    U1 = Tr.LLL_gram()
    U1_inv = U1.inverse()
    return [v*U1_inv for v in cluster0], U1_inv, U1

In [2]:
def MatExp(mat, eps=1*10**-6):
    
    def mat_prod(mat):
        lst = []
        for tup in mat:
            for ele in tup:
                lst.append(ele)
        vec = vector(lst)
        return vec.inner_product(vec)
        
    n = len(mat[0])
    result = matrix.identity(n)
    k = 1
    m = result
    while mat_prod(m) > eps**2:
        m *= 1/k * mat
        result += m
        k += 1
    return result

In [3]:
C = ComplexField(prec = 2000)
P.<x,y,z> = ProjectiveSpace(QQ, 2)
X = P.subscheme(390908548757*x^4 - 1083699236751*x^3*y + 835578482044*x^3*z + 1126610184312*x^2*y^2 -
    1737329379412*x^2*y*z + 669777678687*x^2*z^2 - 520542386163*x*y^3 +
    1204081445939*x*y^2*z - 928398396271*x*y*z^2 + 238611653627*x*z^3 + 90192376558*y^4 -
    278168756247*y^3*z + 321720059816*y^2*z^2 - 165373310794*y*z^3 + 31877479532*z^4)
CR = P.coordinate_ring()
m = matrix(3, 3, [X.defining_polynomials()[0].derivative(CR.gens()[i]).derivative(CR.gens()[j]) for j in range(3) for i in range(3)])
Y = P.subscheme(m.det()) #hessian
R.<u,v> = PolynomialRing(QQ, order='lex')
subs = {x: u, y:v, z:1}
ideal = R.ideal([poly.subs(subs) for poly in X.defining_polynomials()] + [poly.subs(subs) for poly in Y.defining_polynomials()])
polys = ideal.groebner_basis()
Q = P.change_ring(C)
roots2 = [tup[0] for tup in polys[1].univariate_polynomial().change_ring(C).roots()]
roots1 = [-1*polys[0](0, root) for root in roots2]
pts = [Q((roots1[i], roots2[i], 1)) for i in range(len(roots1))]
len(pts)

24

In [5]:
tup = ReduceCluster(pts)

|der| = 681.7391686993723703875093051530378643582775815749394356053090537166556411258102201184484145978538253227559833595106105534456887227811815436701781257136034206072284986505523361996872320650863777331092095389794393001951342346107968210811915603803903376479750436693299079551546878058269392051594630611388713709185175681572573591701766223667433851415885866396639035451189039011587538572241629148244998260354565871691744855094042582270570540667029249535337937525767810123014066272359914809377192696470839569784330034996535176024393299441675435356468974457857030753101562963211928426594304004801370751167792
gain = 556.257309475721172179534092590196318310235951871553922832926982827064788302315754918062281418269754235405456181487662488964486351884807092185881212039605798714975841580626823139414030386681216127784288527529809055228200334438736866862566315234508961696434253429101574877285027295293036170088372434520603599836963024872530432012618053336591472766983925685863130556200205915657929105

|der| = 160.6745376181177299534655136151425976060466493427495746494157462430398269738922794945665591752678774699492740777620851100458791337358039239620180861427582321517702389111696055606376087473141936480651487478857309976650309117455261606729936275042977584679176726031166529501472121203657805755203563416550418295552196846578956194161220528447071462543586396777050434204881652521531732377933655939145960150754572092686604000767416553898363130671647014436988664609117416381381970413910367164439539709526561667828039708762799486375880207204820320094935646153380446427661992295820219942159958886962814789777138
gain = -18.992020231386367432296539613818667621099112549881984821359287995380368314788637177824251394648492084007723653880278587573796125287210423576060181377584159279687712927179208167267927700267571410761538944236102149667492119406409175294485155820283302516019009245128980583828102625211420076781107560063081076004099779214149714140013337490960033452216711330233814636190819012317339914

gain = -12.04454194823160884261961593552593153306713461959137311139773470028365603136199211783625423857093445629500208337199805818633801012338813816649653713154551716991926300572633332309076124289802021997717547271750714295773528540750137483019854137671771775100190095215827960781574625510817077033550494210899809676474682023147472547895920792555323376144732779707038758227934820352730235568309090569549935919624808287017182019821529750340255090518262188225670319428596284920550742252717877953003917481078546397096052146592932612525242734610613843822464507286279736069213871660963759732469910247361301349979658
|der| = 135.24702091418913524213282965405323804530205556941859345432333800069083601611526710543618632343413091113049094880256648217018803510201465064901968125032687665324052117269496128666975212546853738236517088688585981845509940083413893675475249284756417663289118299091013836451561628031187778535726885990022973514863708890100202676573585866815158596021001360108156578680434178406027424

|der| = 0.7811600804714941084469235885428319624273615004608005204571893944631091254289250709156539137750034914113273730133030082639057425404465081276523084154484466561929765179615872681388575080051467934016399443983130658167927156426426308497722032663996273531774912396822546980779229572644348893548076352560948556552874409532732492642131554127278869292628158353276844679603102827652842923878064786688812571017741732005129582531131751964097778067003539247034427404665254698487681147062460985954095224165830733736963359906328777291204823394608430133622270603878197385615540268691077023074643745459148396650738778
gain = -0.025446333945821765873949913020191492991958133662066818586070969663587204448894406442604634614318387612581610795348043392270208132465208207264284491810625547528863168717198804845142859644776149265599237726642406592677813397779122357985424876404619888154428110118896223255731261542865271060237981443337589701237917028484861463007696624002114893307102348905940275514163155132903100

In [12]:
for poly in X.defining_polynomials():
    print(poly(list(tup[2]*vector(P.gens()))))

3*x^4 - 3*x^3*y + x^2*y^2 + 3*y^4 - 3*x^3*z - x*y^2*z + 3*y^3*z - 2*x^2*z^2 - x*y*z^2 + y^2*z^2 + 2*x*z^3 - 3*z^4
